In [147]:
%pip install -r requirements.txt

     ---------------------------------------- 0.0/289.9 kB ? eta -:--:--
     -------------------------------------- 289.9/289.9 kB 8.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [84]:
# poem generation
# Deep learning: a combination of VAE/GAN
# condition the generator of the GAN for rubustness
# latent space!!
# how do you evaluate these?
# discriminator

# test using a model on text and poems to predict whether a given text is a poem or not.
# maybe no input!
# my goal will be to generate 10,000 poems


# future project would be to write books using ai. minimum of 10-50 pages based on the most popular topic on xyz site?
# based on stars guess where the picture was taken in space?

In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import string
import logging
import re
import contractions
import nltk

In [7]:
df = pd.read_pickle('ready_poems.pickle')
print(len(df))

26122


In [11]:
poems = df.content

In [16]:
poems = poems.map(lambda x: nltk.tokenize.word_tokenize(x))

In [26]:
vocab = set([word for poem in poems for word in poem])

In [ ]:
def tf_idf(texts):
    

In [27]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)

In [29]:
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')


In [35]:
ids = ids_from_chars(chars)

In [38]:
ids

<tf.RaggedTensor [[19148, 34574, 104864, 74429, 46874, 96371, 126471],
 [74695, 74625, 127913]]>

In [36]:
out = chars_from_ids(ids)

In [37]:
out

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
# remove translated from/by

In [ ]:
# replace \n with newline
# expand contractions
# split into tokens
# convert into lowecase
# Decoding Unicode characters into a normalized form, such as UTF8???
# remove numbers???
# remove all punctuation
# filter out non alphabetic
# recontruct into poems
# us isalpha
# string = re.sub(r"[,.”;@—#“…?!&$-’_)–(*)]+", ' ', string)
# string = re.sub('\s\s+' , ' ', cleaner)

In [18]:
# log the devices being used for tensorflow operations
tf.debugging.set_log_device_placement(True)